This notebook illustrates the use of estimator and VQD runtime.

Author: Tomasz Stopa

Modified by: Gregoire Cattan

In [1]:
!pip install qiskit==0.45.1
!pip install qiskit_ibm_runtime==0.17.0
!pip install qiskit_ibm_provider


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\alaga\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\alaga\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\alaga\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
from qiskit.opflow import Z, I, X, Y, H
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.random import random_circuit
from qiskit.circuit.library import TwoLocal, RealAmplitudes, EfficientSU2

from qiskit_ibm_runtime import Estimator
from qiskit.algorithms.eigensolvers import NumPyEigensolver
from qiskit.algorithms.optimizers import SPSA, SLSQP
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator
from qiskit.primitives import Sampler, Estimator
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit.algorithms.eigensolvers import VQD

## Basic (local) usage of Estimator primitive

In [3]:
# Define operator and circuit
operator = SparsePauliOp("XY")
state_circuit = random_circuit(2, 2).decompose(reps=1)
estimator = Estimator()

In [4]:
# Run estimator
job = estimator.run(state_circuit, operator)
job.result()

EstimatorResult(values=array([0.]), metadata=[{}])

## Estimator can accept multiple circuit+operator pairs:

In [5]:
circuits = (
    random_circuit(2, 2, seed=0).decompose(reps=1),
    random_circuit(2, 2, seed=0).decompose(reps=1),
)
observables = (
    SparsePauliOp("XZ"),
    SparsePauliOp("XX"),
)

In [6]:
job = estimator.run(circuits, observables)
job.result()

EstimatorResult(values=array([0.50006466, 0.25264472]), metadata=[{}, {}])

## Estimator as Qiskit Runtime program

In [8]:
service = QiskitRuntimeService(channel="ibm_quantum")

with Session(service=service, backend="ibmq_qasm_simulator") as session:
    estimator = Estimator(session=session)
    job = estimator.run(state_circuit, operator)
    result = job.result()

print(result.values[0])

TypeError: Estimator.__init__() takes 1 positional argument but 2 were given

## Estimator for parametrized circuit

In [ ]:
parametrized_circuit = RealAmplitudes(num_qubits=2, reps=2).decompose(reps=1)
parametrized_circuit.draw()

┌──────────┐     ┌──────────┐     ┌──────────┐
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├──■──┤ Ry(θ[4]) ├
     ├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ X ├┤ Ry(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├
     └──────────┘└───┘└──────────┘└───┘└──────────┘

In [ ]:
print(parametrized_circuit.num_parameters)

6


In [ ]:
parameter_values = [0, 1, 2, 3, 4, 5]

In [ ]:
with Session(service=service, backend="ibmq_qasm_simulator") as session:
    estimator = Estimator(session)
    job = estimator.run(parametrized_circuit, operator, parameter_values)
    result = job.result()

result.values

array([0.])

## Qiskit Runtime VQD

In [12]:
from qiskit.primitives import Estimator

vqd = VQD(
    estimator=Estimator(),
    fidelity=ComputeUncompute(sampler=Sampler()),
    ansatz=TwoLocal(2, rotation_blocks=["ry", "rz"], entanglement_blocks="cz", reps=1),
    optimizer=SLSQP(),
    k=4,
    betas=[6, 6, 6],
)

In [27]:
result = vqd.compute_eigenvalues(operator=X ^ X + Z ^ Z)
vqd_values = result.optimal_values
print(vqd_values)

[-1.41421351 -1.41421346 -1.41421006 -1.41354047]


## Exercise 8.1
Given the following Hamiltonian:

In [28]:
H4 = SparsePauliOp.from_list(
    [
        ("IIII", -1.0),
        ("IZZI", 0.4),
        ("ZIII", -0.4),
        ("ZZZZ", -0.1),
        ("XXIX", 0.1),
        ("XXXX", 2.0),
        ("IIIX", 0.5),
    ]
)

- Use VQD algorithm to calculate first 6 energies of a system
- Compare the energies with classicaly calculated ones
- Analyze the ansatz - draw it and check how many parameters it needs to have to give reasonable accuracy of energies.

In [37]:
ansatz = TwoLocal(4, rotation_blocks=["ry", "rz"], entanglement_blocks="cz", reps=1)

vqd = VQD(
    estimator=Estimator(),
    fidelity=ComputeUncompute(sampler=Sampler()),
    ansatz=ansatz,
    optimizer=SLSQP(),
    k=6,
    betas=[6, 6, 6, 6, 6],
)

In [38]:
result = vqd.compute_eigenvalues(operator=H4)
vqd_values = result.optimal_values
print(vqd_values)

[-3.95155742 -3.95022134 -3.1278432  -2.95156101 -3.12784371 -2.95046278]


In [41]:
classical_eigenvalues = (
    NumPyEigensolver(k=6).compute_eigenvalues(operator=H4).eigenvalues
)
print(classical_eigenvalues)

[-3.96118304 -3.96118304 -3.14009098 -3.14009098 -2.93932096 -2.93932096]


In [42]:
ansatz.draw()

»
q_0: »
     »
q_1: »
     »
q_2: »
     »
q_3: »
     »
«     ┌──────────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                                 ├
«     │                                                                                                  │
«q_1: ┤1                                                                                                 ├
«     │  TwoLocal(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5],θ[6],θ[7],θ[8],θ[9],θ[10],θ[11],θ[12],θ[13],θ[14],θ[15]) │
«q_2: ┤2                                                                                                 ├
«     │                                                                                                  │
«q_3: ┤3                                                                                                 ├
«     └──────────────────────────────────────────────────────────────────────────────────────────────────┘

## Homework
Read the following Qiskit tutorial on VQE in general and it's application to simulate simple molecules in particular:

https://github.com/Qiskit/textbook/blob/main/notebooks/ch-applications/vqe-molecules.ipynb